In [103]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
import pandas as pd
import imageio

%matplotlib inline

In [104]:
# labels = np.genfromtxt('labels.csv', delimiter=',')
labels = pd.read_csv('labels.csv', float_precision='road_trip')
labels.head(5)

,Unnamed: 0,img_name,base,base color R,base color G,base color B,specular,specular color R,specular color G,specular color B,specular roughness,metalness
0,0,0.bmp,0.196696,[0.02551167982996516,0.523569,0.09533690194549593],0.903675,[0.9414576345001383,0.927580,0.418087362819755],0.120439,0.941755
1,1,1.bmp,0.563387,[0.13560656102894886,0.715835,0.6311335628961977],0.381407,[0.9228773407900674,0.328530,0.294482323738513],0.439152,0.511734
2,2,2.bmp,0.433630,[0.06292513279208145,0.468129,0.49781547380782],0.622876,[0.8260393377700712,0.115993,0.1549741111822439],0.646591,0.490058
3,3,3.bmp,0.471443,[0.8909351516333082,0.133360,0.7187711144029106],0.419356,[0.08579008506320773,0.466882,0.36951439970192435],0.154928,0.632136
4,4,4.bmp,0.172493,[0.1089380911674539,0.179026,0.4079338804909298],0.893968,[0.9620524644071147,0.906355,0.3523653506316685],0.708425,0.002353


In [105]:
labels["base color R"] = labels["base color R"].str[1:]
labels["base color B"] = labels["base color B"].str[:-1]
labels["specular color R"] = labels["specular color R"].str[1:]
labels["specular color B"] = labels["specular color B"].str[:-1]
labels["img_name"] = labels["img_name"].str[:-3]+"jpg"

In [106]:
labels.head(5)

,Unnamed: 0,img_name,base,base color R,base color G,base color B,specular,specular color R,specular color G,specular color B,specular roughness,metalness
0,0,0.jpg,0.196696,0.02551167982996516,0.523569,0.09533690194549593,0.903675,0.9414576345001383,0.927580,0.418087362819755,0.120439,0.941755
1,1,1.jpg,0.563387,0.13560656102894886,0.715835,0.6311335628961977,0.381407,0.9228773407900674,0.328530,0.294482323738513,0.439152,0.511734
2,2,2.jpg,0.433630,0.06292513279208145,0.468129,0.49781547380782,0.622876,0.8260393377700712,0.115993,0.1549741111822439,0.646591,0.490058
3,3,3.jpg,0.471443,0.8909351516333082,0.133360,0.7187711144029106,0.419356,0.08579008506320773,0.466882,0.36951439970192435,0.154928,0.632136
4,4,4.jpg,0.172493,0.1089380911674539,0.179026,0.4079338804909298,0.893968,0.9620524644071147,0.906355,0.3523653506316685,0.708425,0.002353


In [107]:
vector_values = labels.iloc[:,2:]
vector_values = vector_values.astype(float)
vector_values.head(5)

,base,base color R,base color G,base color B,specular,specular color R,specular color G,specular color B,specular roughness,metalness
0,0.196696,0.025512,0.523569,0.095337,0.903675,0.941458,0.927580,0.418087,0.120439,0.941755
1,0.563387,0.135607,0.715835,0.631134,0.381407,0.922877,0.328530,0.294482,0.439152,0.511734
2,0.433630,0.062925,0.468129,0.497815,0.622876,0.826039,0.115993,0.154974,0.646591,0.490058
3,0.471443,0.890935,0.133360,0.718771,0.419356,0.085790,0.466882,0.369514,0.154928,0.632136
4,0.172493,0.108938,0.179026,0.407934,0.893968,0.962052,0.906355,0.352365,0.708425,0.002353


In [108]:
np_labels = vector_values.values
np_labels[0]

array([0.19669609, 0.02551168, 0.52356934, 0.0953369 , 0.90367509,
       0.94145763, 0.92757973, 0.41808736, 0.1204395 , 0.94175506])

THE BELOW CELL TAKES AGES TO RUN. ONLY RUN IT ONCE.

In [ ]:
imagedir = './'
images = np.array([])
for i in range(4000):
    fname = "images/" + str(i) + ".jpg"
    img = imageio.imread(fname)
    images = np.append(images, img)

In [ ]:
images[0]

In [ ]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    s = 1.0 / (1.0 + np.exp(-z))
    return s

In [ ]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """    
    w = np.zeros((dim, 1))
    b = 0

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [ ]:
# GRADED FUNCTION: propagate

def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    ### START CODE HERE ### (≈ 2 lines of code)
    A = sigmoid(np.dot(w.T, X) + b)                                    # compute activation
    cost = (-1.0 / m) * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))                              # compute cost
    ### END CODE HERE ###
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    ### START CODE HERE ### (≈ 2 lines of code)
    dw = (1.0 / m) * np.dot(X, (A - Y).T)
    db = (1.0 / m) * np.sum(A - Y)
    ### END CODE HERE ###

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [ ]:
# GRADED FUNCTION: optimize

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        ### START CODE HERE ### 
        grads, cost = propagate(w, b, X, Y)
        ### END CODE HERE ###
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        ### START CODE HERE ###
        w = w - learning_rate*dw
        b = b - learning_rate*db
        ### END CODE HERE ###
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [ ]:
# GRADED FUNCTION: predict

def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    ### START CODE HERE ### (≈ 1 line of code)
    A = sigmoid(np.dot(w.T, X) + b)
    ### END CODE HERE ###
    
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        ### START CODE HERE ### (≈ 4 lines of code)
        Y_prediction[0][i] = 1 if (A[0][i] > 0.5) else 0
        ### END CODE HERE ###
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [ ]:
# GRADED FUNCTION: model

def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
    ### START CODE HERE ###
    
    # initialize parameters with zeros (≈ 1 line of code)
    w, b = initialize_with_zeros(X_train.shape[0])

    # Gradient descent (≈ 1 line of code)
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples (≈ 2 lines of code)
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    ### END CODE HERE ###

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d